In [1]:
%cd "/content/drive/MyDrive/Colab Notebooks/DeepLearning 2021/ex3/newVer/VAE_SemiSupervised"
# %pip install pytorch_lightning

import yaml
import torch
from experiment import VAEXperiment
import torch.backends.cudnn as cudnn
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from svm.utils import SVMClass

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
with open("configs/vae.yaml", 'r') as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

/content/drive/MyDrive/Colab Notebooks/DeepLearning 2021/ex3/newVer/VAE_SemiSupervised


In [2]:
%run run.py

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


======= Training VanillaVAE =======



  | Name  | Type       | Params
-------------------------------------
0 | model | VanillaVAE | 3.2 M 
-------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.968    Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/drive/My Drive/Colab Notebooks/DeepLearning 2021/ex3/newVer/VAE_SemiSupervised/logs/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Training is done.
======= Finished Training =======


## SVM Classifier & Testing

In [3]:
import pandas as pd
# num_samples = [100, 600, 1000, 3000]
num_samples = [100, 600, 1000, 3000]
df = pd.DataFrame(columns=['Samples_num', 'Accuracy', 'Loss'])
for samples in num_samples:
  model = torch.load(config['logging_params']['best_model_dir'], map_location=device)
  experiment = VAEXperiment(model, config['exp_params'], config['logging_params'])
  classifier = SVMClass(model, config['exp_params'])
  train_dataset = experiment.datasets[0]
  test_dataset = experiment.test_dataset
  latent, labels = classifier.gen_latent(train_dataset, samples)
  classifier.train(latent, labels)
  classifier.test(test_dataset)
  print(f"Classifier trained with:{samples} samples, Resulted Accuracy:{100*classifier.accuracy:.0f}%,"
        f" Loss:{classifier.loss:.2f}")
  df = df.append([samples, 100*classifier.accuracy, classifier.loss])
df.to_csv('results.csv',index=False)

Classifier trained with:100 samples, Resulted Accuracy:73%, Loss:0.00
Classifier trained with:600 samples, Resulted Accuracy:74%, Loss:0.00


RuntimeError: ignored

## Test for 3000 labeled samples